In [2]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import joblib
import optuna

In [12]:
# precios de casas
data = fetch_california_housing()
X = data.data
y = data.target

### Attribute Information:
  - MedInc:        median income in block group
  - HouseAge:      median house age in block group
  - AveRooms:      average number of rooms per household
  - AveBedrms:     average number of bedrooms per household
  - Population:    block group population
  - AveOccup:      average number of household members
  - Latitude:      block group latitude
  - Longitude:     block group longitude

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# función para entrenar y evaluar un modelo con hiperparámetros dados
def train_evaluate_model(params, X_train, y_train, X_test, y_test):
    model = RandomForestRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [14]:
df=pd.DataFrame(np.c_[data['data'], data['target']],
                  columns= np.append(data['feature_names'], ['target']))
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


El objetivo es emplear las técnicas de búsqueda de hiperparámetros, NO hacer un análisis exploratorio. Tener en cuenta que primero hay que hacer ese análisis.

In [19]:
# grid search

grid_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), grid_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_grid_params = grid_search.best_params_
best_grid_model = RandomForestRegressor(**best_grid_params, random_state=42)
best_grid_model.fit(X_train, y_train)
grid_search_mse = mean_squared_error(y_test, best_grid_model.predict(X_test))

Esto crea un objeto **GridSearchCV**, que:

* Busca entre todas las combinaciones posibles de hiperparámetros especificadas en grid_params.
* Usa validación cruzada con 5 folds (cv=5).
* Optimiza la métrica neg_mean_squared_error (error cuadrático medio negativo, porque GridSearchCV maximiza la métrica por defecto).
* Usa todos los núcleos del procesador (n_jobs=-1).

**grid_search.fit**: entrena todos los modelos posibles combinando los parámetros de grid_params, usando validación cruzada, y guarda el que tuvo mejor desempeño promedio.

In [15]:
# random search
random_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4]
}

random_search = RandomizedSearchCV(RandomForestRegressor(random_state=42), random_params, n_iter=4, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
random_search.fit(X_train, y_train)
best_random_params = random_search.best_params_
best_random_model = RandomForestRegressor(**best_random_params, random_state=42)
best_random_model.fit(X_train, y_train)
random_search_mse = mean_squared_error(y_test, best_random_model.predict(X_test))

Este código está definiendo un Randomized Search para encontrar los mejores hiperparámetros de un modelo RandomForestRegressor.
Es una técnica de búsqueda aleatoria de combinaciones de hiperparámetros.

A diferencia de GridSearchCV (que prueba todas las combinaciones), esta prueba un número fijo de combinaciones al azar (en este caso, 4).
Parametros:
* RandomForestRegressor(random_state=42): El modelo que usa
* random_params: Diccionario con las posibles combinaciones de hiperparámetros.
* n_iter=4: Va a probar solo 4 combinaciones distintas (elegidas aleatoriamente de random_params).
* cv=5: Usa validación cruzada de 5 pliegues para evaluar cada combinación.
* scoring='neg_mean_squared_error': La métrica que se usa es el error cuadrático medio negativo (scikit-learn multiplica por -1 porque quiere maximizar los scores).
* n_jobs=-1: Usa todos los núcleos del procesador disponibles para paralelizar el proceso y hacerlo más rápido.

In [17]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 200)
    max_depth = trial.suggest_int('max_depth', 5, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)

    params = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf
    }

    mse = train_evaluate_model(params, X_train, y_train, X_test, y_test)
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
best_optuna_params = study.best_params
best_optuna_model = RandomForestRegressor(**best_optuna_params, random_state=42)
best_optuna_model.fit(X_train, y_train)
optuna_mse = mean_squared_error(y_test, best_optuna_model.predict(X_test))


[I 2025-05-22 14:49:57,225] A new study created in memory with name: no-name-df21136e-7675-4168-8ca4-8df28e7c7c1f
[I 2025-05-22 14:50:14,593] Trial 0 finished with value: 0.2597375521317425 and parameters: {'n_estimators': 123, 'max_depth': 17, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.2597375521317425.
[I 2025-05-22 14:50:30,006] Trial 1 finished with value: 0.26072164798759817 and parameters: {'n_estimators': 113, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.2597375521317425.
[I 2025-05-22 14:50:44,257] Trial 2 finished with value: 0.2819152055619556 and parameters: {'n_estimators': 117, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.2597375521317425.
[I 2025-05-22 14:51:06,632] Trial 3 finished with value: 0.2814012300934095 and parameters: {'n_estimators': 197, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with valu

Este código utiliza Optuna, una biblioteca de optimización automática de hiperparámetros, para encontrar la mejor combinación posible de parámetros para un modelo RandomForestRegressor, minimizando el error cuadrático medio (MSE).

**trial.suggest_int** le dice a Optuna: Elegí un valor entero para n_estimators entre 100 y 200 (por ejemplo, 157), y usalo en este ensayo (trial) de la optimización.

**study.optimize**: Crea un estudio de Optuna para minimizar la métrica (el MSE) y ejecuta la búsqueda durante 10 pruebas distintas (con 10 combinaciones de hiperparámetros).

**best_optuna_param**: Recupera los mejores hiperparámetros encontrados y entrena el modelo definitivo con esos parámetros.

**optuna_mse**: Predice sobre el set de prueba y calcula el MSE final con los mejores hiperparámetros.

**Ventajas**:
* Más eficiente: Optuna usa una estrategia de búsqueda inteligente (tipo bayesiana).

* Más flexible: Podés optimizar cualquier métrica, usar condiciones, early stopping, etc.

* Más rápido: En muchos casos necesita menos pruebas que un grid para encontrar buenos resultados.

In [20]:
best_grid_params

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [16]:
best_random_params

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_depth': 20}

In [18]:
best_optuna_params

{'n_estimators': 122,
 'max_depth': 20,
 'min_samples_split': 4,
 'min_samples_leaf': 2}

In [21]:
print("Grid Search MSE:", grid_search_mse)
print("Random Search MSE:", random_search_mse)
print("Optuna MSE:", optuna_mse)

Grid Search MSE: 0.2545922861286096
Random Search MSE: 0.26029181713291216
Optuna MSE: 0.25474464085627135
